# Retrieving the time lag between P-wave arrival and S-wave arrival in seismic recordings of tectonic tremor

## Looking at all the tremor from the same location

One problem with the pevious method is that the source of the tremor is moving with time. Now, if we look at all the one-minute-long time windows where the source of the tremor is located at about the same given location, the time lag between the P-wave and the S-wave should always be the same.

If we compute the cross correaltion of the vertical and the horizontal component, and then we stack over all the tremor windows, we should obtain a clear signal.

That what we are going to do now.

We begin by importing the Python modules that we will need during the analysis.

In [1]:
import obspy
import obspy.clients.earthworm.client as earthworm
import obspy.clients.fdsn.client as fdsn
from obspy import read_inventory
from obspy import UTCDateTime
from obspy.core.stream import Stream
from obspy.signal.cross_correlation import correlate

import matplotlib.pyplot as plt
import numpy as np
import pickle
import sys

from scipy.io import loadmat
from math import pi, cos, sin, sqrt

from date import matlab2ymdhms
from stacking import linstack, powstack, PWstack

%matplotlib inline

We load the tremor catalog from A. Ghosh to know where and when we have tremor.

In [2]:
data = loadmat('../data/timelags/mbbp_cat_d_forHeidi')
mbbp_cat_d = data['mbbp_cat_d']
lat = mbbp_cat_d[:, 2]
lon = mbbp_cat_d[:, 3]

We choose to look at all the tremor located in a 5 km by 5 km square cell, centered around the array.

In [3]:
# Size and location of cell
ds = 5.0
x0 = 0.0
y0 = 0.0
# Location of the center of the array
lat0 = 47.95728
lon0 = -122.92866
# Formula to convert degrees in latitude and longitude into kilometers
a = 6378.136
e = 0.006694470
dx = (pi / 180.0) * a * cos(lat0 * pi / 180.0) / sqrt(1.0 - e * e * sin(lat0 * pi / 180.0) * sin(lat0 * pi / 180.0))
dy = (3.6 * pi / 648.0) * a * (1.0 - e * e) / ((1.0 - e * e * sin(lat0 * pi / 180.0) * sin(lat0 * pi / 180.0)) ** 1.5)
# Latitude and longitude of the edges of the square cell
lonmin = lon0 + (x0 - 0.5 * ds) / dx
lonmax = lon0 + (x0 + 0.5 * ds) / dx 
latmin = lat0 + (y0 - 0.5 * ds) / dy
latmax = lat0 + (y0 + 0.5 * ds) / dy

We now look at the time when we have tremor in this square cell.

In [4]:
find = np.where((lat >= latmin) & (lat <= latmax) & (lon >= lonmin) & (lon <= lonmax))
tremor = mbbp_cat_d[find, :][0, :, :]
nt = np.shape(tremor)[0]
print(nt)

70


Tremor with a source located in this square cell were recorded during 70 minutes. That should be enough to have a good signal on the stack of the cross correlations. We are now going to choose one of the tremor window, and look at what the cross correlation look like.

In [5]:
i = 0

We need to convert the beginning and end time of the time window we are interested in from the Matlab format to the Python format.

In [6]:
(YY1, MM1, DD1, HH1, mm1, ss1) = matlab2ymdhms(tremor[i, 0])
t1 = UTCDateTime(str(YY1) + '-' + str(MM1) + '-' + str(DD1) + 'T' + str(HH1) + ':' + str(mm1) + ':' + str(ss1))
(YY2, MM2, DD2, HH2, mm2, ss2) = matlab2ymdhms(tremor[i, 1])
t2 = UTCDateTime(str(YY2) + '-' + str(MM2) + '-' + str(DD2) + 'T' + str(HH2) + ':' + str(mm2) + ':' + str(ss2))

We are going to download 10 additional seconds of data before and after, in order to taper the signal.

In [7]:
Tstart = t1 - 10.0
Tend = t2 - 10.0